## Getting Reddit Comment Data

In [1]:
import praw
from praw.models import MoreComments
import tqdm
import pandas as pd
import statistics
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
reddit = praw.Reddit(
    client_id="p1dG7hgoowK4BSlUdar1WQ",
    client_secret="pEePtSnw7KMDZi6fCzkKaOth_pgKpQ",
    password="outdoortuesday",
    user_agent="Big Data by u/DISWillJayminMaya ",
    username="DISWillJayminMaya ",
)

In [3]:
analyzer = SentimentIntensityAnalyzer()

def returnListofScores(listofcomments):
    templist=[]
    for com in listofcomments:
        temp = analyzer.polarity_scores(com)
        compScore = temp['compound']
        templist.append(compScore)
    return templist


In [4]:
tickerlist = ['GME', 'Gamestop', 'SPY', 'TWTTR', 'Twitter', 'TSLA', 'Tesla', 'AMD']
subreddit = reddit.subreddit('wallstreetbets')

scoreDict={key:list() for key in tickerlist}

hot = subreddit.hot(limit=25) # getting first 15 posts in the 'hot' section of the subreddit
sum = [0] * len(tickerlist) # our output array
counttotal = 0 # total number of comment read
submissions_counter = 0

rel_comments = [] # List of comments that are relevant to the ticker list items

In [5]:
# Loop for fetching the comments and the amount of time each ticker is mentioned
for submissions in hot:
    if not submissions.stickied:
        submissions_counter+=1
        if submissions_counter > 5:
            comments = submissions.comments
            for comment in comments:
                if isinstance(comment, MoreComments):
                    continue
                counttotal+=1
                for i, ticker in enumerate(tickerlist):
                    if ticker in comment.body:
                        rel_comments.append(comment.body)
                        sum[i]=sum[i]+1
                        temp = returnCompScore(comment.body)
                        scoreDict[ticker].append(temp)

NameError: name 'returnCompScore' is not defined

In [6]:
meanDict={key:list() for key in tickerlist}
spreadDict={key:list() for key in tickerlist}

for tick in tickerlist:
    if len(scoreDict[tick])>0:
        meanDict[tick]=statistics.mean(scoreDict[tick])
    if len (scoreDict[tick])>1:
        spreadDict[tick]=statistics.stdev(scoreDict[tick])

In [7]:
print(meanDict)
print(spreadDict)

{'GME': [], 'Gamestop': [], 'SPY': [], 'TWTTR': [], 'Twitter': [], 'TSLA': [], 'Tesla': [], 'AMD': []}
{'GME': [], 'Gamestop': [], 'SPY': [], 'TWTTR': [], 'Twitter': [], 'TSLA': [], 'Tesla': [], 'AMD': []}


In [8]:
output=pd.DataFrame(data={'Tick': tickerlist, 'Counts': sum})
print('Total comments read: ',counttotal)
print(output[output['Counts']>0])

Total comments read:  249
  Tick  Counts
0  GME       1


## Getting Stock Information

In [9]:
import requests, os, json, csv

size = 'compact' # 'full' for complete historical data, 'compact' for most recent 100
ticker = ['GME', 'SPY', 'TWTTR', 'TSLA', 'AMD'] # stock tickers to search for
datatype = 'csv' # 'json' for JSON output, 'csv' for CSV output

for stock in ticker:
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={stock}&outputsize={size}&datatype={datatype}&apikey=QC1C7LRPUTLC597Q'
    response = requests.get(url)
    #Save CSV to file
    with open(f'{stock}.csv', 'wb') as file:
        file.write(response.content)

## Charting Data

In [10]:
amd_data = pd.read_csv("AMD.csv")
amd_data

,timestamp,open,high,low,close,volume
0,2022-04-28,86.670,90.580,84.7800,89.64,91495449
1,2022-04-27,84.250,87.900,84.0200,84.91,83125054
2,2022-04-26,89.740,90.120,85.0800,85.16,87805574
3,2022-04-25,89.860,91.370,88.6100,90.69,93481042
4,2022-04-22,90.030,91.460,87.9350,88.14,75017652
...,...,...,...,...,...,...
95,2021-12-10,141.290,141.365,135.8200,138.55,42224275
96,2021-12-09,145.160,146.690,137.8000,138.10,53019926
97,2021-12-08,144.960,147.040,142.7000,145.24,40977478
98,2021-12-07,143.900,145.760,141.0001,144.85,53359432


get avg, spread, median of sentiment of comments in any given period
    compare this with the performance of the stock

model:
    based on last x days of reddit comments, what is the price going to be?
    take data from x days, put it all into one vector, and predict from this
    
    

## Making a function to return all comments that mention a stock based on a given date range

In [11]:
import math
import json
import requests
import itertools
import numpy as np
import time
from datetime import datetime, timedelta

def make_request(uri, max_retries = 5):
    """
    Function taken from medium article:
    https://medium.com/@pasdan/how-to-scrap-reddit-using-pushshift-io-via-python-a3ebcc9b83f4
    """
    def fire_away(uri):
        response = requests.get(uri)
        assert response.status_code == 200
        return json.loads(response.content)
    current_tries = 1
    while current_tries < max_retries:
        try:
            time.sleep(1)
            response = fire_away(uri)
            return response
        except:
            time.sleep(1)
            current_tries += 1
    return fire_away(uri)

def get_intervals(startPOSTIX, endPOSTIX, daysInInterval = 1):
    """
    get_intervals goes day by day through the start and end dates, returning that day's POSTIX
    """
    # 86,400 seconds in a day:
    period = (86400 * daysInInterval)
    end = startPOSTIX + period
    
    yield(int(startPOSTIX), int(end))
    
    padding = 1
    while end <= endPOSTIX:
        startPOSTIX = end + padding
        end = (startPOSTIX - padding) + period
        yield int(startPOSTIX), int(end)
    
    
def pull_posts_for(subreddit, start_at, end_at):
    """
    Function taken from medium article:
    https://medium.com/@pasdan/how-to-scrap-reddit-using-pushshift-io-via-python-a3ebcc9b83f4
    """
    def map_posts(posts):
        return list(map(lambda post: {
            'id': post['id'],
            'created_utc': post['created_utc'],
            'prefix': 't4_'
        }, posts))
    
    SIZE = 500
    URI_TEMPLATE = r'https://api.pushshift.io/reddit/search/submission?subreddit={}&after={}&before={}&size={}'
    
    post_collections = map_posts(
        make_request(URI_TEMPLATE.format
                     (subreddit, start_at, end_at, SIZE))['data'])
    n = len(post_collections)
    while n == SIZE:
        last = post_collections[-1]
        new_start_at = last['created_utc'] - (10)
        
        more_posts = map_posts( \
            make_request( \
                URI_TEMPLATE.format( \
                    subreddit, new_start_at, end_at, bSIZE))['data'])
        
        n = len(more_posts)
        post_collections.extend(more_posts)
    return post_collections

def get_comments_by_date (startDate, endDate, subreddit='wallstreetbets'):
    """
    Takes a given time interval and scrapes the given subreddit for all of the comments
    that relate to the given ticker name, returning them as an array. Basic structure taken 
    from medium article.
    """
    # Converting start and end dates to POSTIX:
    startDate = math.floor(startDate.timestamp())
    endDate = math.floor(endDate.timestamp())

    posts = []
    # This loop gets all of the posts in the given timeframe
    for interval in get_intervals(startDate, endDate):
        print(" -- Fetching Posts From: ", datetime.fromtimestamp(interval[0]), " to ", datetime.fromtimestamp(interval[1]))
        pulled_posts = pull_posts_for(subreddit, interval[0], interval[1])
        posts.extend(pulled_posts)
        time.sleep(.100) # So as not to over request reddit
    
    TIMEOUT_SECS = .100
    
    reddit_posts = []
    reddit_comments = {}
    reddit_comments[startDate] = []
    # Going through each unique post and comment and adding them to the relevant arrays
    for sub_id in np.unique([post['id'] for post in posts]):
        sub = reddit.submission(id=sub_id)
        reddit_posts.append(sub)
        sub.comments.replace_more(limit=None)
        # Looping through each comment:
        for comment in sub.comments.list(): 
            reddit_comments[startDate].append(comment.body)
                
            time.sleep(TIMEOUT_SECS)
    
    return reddit_comments

In [12]:
start = datetime.today() - timedelta(days=0.5)
end = datetime.today()

all_comments = get_comments_by_date(start, end)

 -- Fetching Posts From:  2022-04-29 01:18:32  to  2022-04-30 01:18:32


In [43]:
tickerlist = ['GME', 'Gamestop', 'SPY', 'TWTTR', 'Twitter', 'TSLA', 'Tesla', 'AMD']
ticker_dict = {}

for tick in tickerlist:
    ticker_dict[tick] = {}
    
for tick in tickerlist:
    for key in all_comments.keys():
        ticker_dict[tick][key] = []
        
for tick in tickerlist:
    for key in all_comments.keys():
        for com in all_comments[key]:
            if tick in com:
                ticker_dict[tick][key].append(com)

In [48]:
ticker_dict['TSLA']

{1651187912: ['So in other words, long $TSLA',
  'So wait… are you saying calls or puts on TSLA?',
  'All I heard was buy TSLA',
  'So..buy TSLA or die?',
  'shitpost, conspiracy, or the gift of prophecy? only time will tell. positions: TSLA, TWTR puts',
  'Sooooo…. TSLA is a buy again?',
  'While this is a joke post, it’s probably true. He’s said the problem with self driving cars is that you need to solve real world AI first, which is where the robot comes in. The data he can acquire from Twitter could propel that AI in the real world to new heights. \n\nI was trying to figure out the play aside from free speech and now I think you have.\n\nBullish TSLA',
  'So… $TSLA calls it is.',
  ' The kind of shitposting that I can get behind.  Puts on TSLA it is because surely Skynet will kill off its maker before anything else.',
  'Our only hope is for Jim Cramer to say "Buy $TSLA!!"',
  'Sir you are at 69 likes, i am not gonna press like that comment to honor the king of the numbers\n\nEdit

## Making the Comments Data Frame

- Get dates that the comments are from
    - two cols, start and end date
- Get all of the comments from the dates
    - Get the count, and sentiment data from this
- Should each ticker have its own df?

In [ ]:
comment_df = pd.DataFrame()
tickerlist = ['GME', 'Gamestop', 'SPY', 'TWTTR', 'Twitter', 'TSLA', 'Tesla', 'AMD']
# all_comments = get_comments_by_date(start, end)
ticker_dates_dict = {}

for tick in tickerlist:
    ticker_dates_dict[tick] = pd.DataFrame()

# Dates begin yesterday through today:
end_day = datetime.today() # End day is today
start_day = end_day - timedelta(days=7) # Start day is one week ag

# Running for the past week:
for tick in tickerlist:
    for i in range(0, 7):
        end_day = datetime.today() - timedelta(days=i)
        start_day = datetime.today() - timedelta(days=i + 1)

In [ ]:
ticker_dates_dict['SPY']